### Import the packages

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

2022-02-20 09:51:53.626525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-20 09:51:53.626543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Define the features for the model

In [2]:
num_classes = 2
batch_size = 32
num_epochs = 20
img_w = 115
img_h = 160
img_size = 160
log_dir = './log'
shuffle_buffer_size = 1024
img_channels = 3

### Create functions for preprocessing and augmentation of the data

In [3]:
def preprocess(ds):
    x = tf.image.resize_with_pad(ds['image'], img_size, img_size)
    x = tf.cast(x, tf.float32)
    x = (x/127.5) - 1
    return x, ds['label']

In [4]:
def augmentation(image, label):
    image = tf.image.random_brightness(image, .1)
    image = tf.image.random_contrast(image, lower = 0.0, upper = 1.0)
    image = tf.image.random_flip_left_right(image)
    return image, label

### Create a function to get the dataset and break into train, validation and test sets

In [5]:
def get_dataset(dataset_name):
    train, info_train = tfds.load(dataset_name, split = 'train[:80%]', with_info = True)
    val, info_val = tfds.load(dataset_name, split = 'train[80%:90%]', with_info = True)
    test, info_test = tfds.load(dataset_name, split = 'train[90%:]', with_info = True)

    num_classes = info_train.features['label'].num_classes
    assert num_classes >= info_val.features['label'].num_classes
    num_examples = info_train.splits['train'].num_examples * 0.9
    img_h, img_w, img_channels = info_train.features['image'].shape
    
    train = train.map(preprocess).shuffle(shuffle_buffer_size).batch(batch_size).cache().repeat()
    train = train.map(augmentation)
    train = train.prefetch(tf.data.experimental.AUTOTUNE)
    
    val = val.map(preprocess).batch(batch_size).cache().repeat()
    val = val.prefetch(tf.data.experimental.AUTOTUNE)
    
    test = test.map(preprocess).batch(batch_size)
    test = test.prefetch(tf.data.experimental.AUTOTUNE)
    
    return train, info_train, val, info_val, test, info_test, img_h, img_w, img_channels, num_classes, num_examples

In [6]:
train, info_train, val, info_val, test, info_test, img_h, img_w, img_channels, num_classes, num_examples = get_dataset('malaria')

2022-02-20 09:51:56.414186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-20 09:51:56.414211: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-20 09:51:56.414225: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (guilherme-dell): /proc/driver/nvidia/version does not exist
2022-02-20 09:51:56.414612: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Create the function for the CNN model defining the layers

This is a very common structure for CNN using the Keras Sequential class

In [7]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (img_size, img_size, img_channels))
        , tf.keras.layers.MaxPool2D(pool_size = (2, 2))
        , tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu')
        , tf.keras.layers.MaxPool2D(pool_size = (2, 2))
        , tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu')
        , tf.keras.layers.Dropout(rate = 0.3)
        , tf.keras.layers.Flatten()
        , tf.keras.layers.Dense(128, activation = 'relu')
        , tf.keras.layers.Dropout(rate = 0.3)
        , tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ])
    return model

### Create the function to parse the train and val tests, compile and fit the model as defined in the previous function

In [8]:
def compile_fit(train, val, learning_rate):
    model = create_model()
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
                  , loss = 'sparse_categorical_crossentropy'
                  , metrics = ['accuracy'])
    
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy'
                                                          , min_delta = 0.0001
                                                          , patience = 4)
    
    model.fit(train
              , epochs = num_epochs
              , steps_per_epoch = int(num_examples/batch_size)
              , validation_data = val
              , validation_steps = 1
              , validation_freq = 1
              , callbacks = [earlystop_callback])
    return model

In [9]:
model = compile_fit(train, val, 0.001)

Epoch 1/20


2022-02-20 09:51:58.305028: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102252544 exceeds 10% of free system memory.
2022-02-20 09:51:58.683583: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 34151040 exceeds 10% of free system memory.
2022-02-20 09:51:58.976218: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51126272 exceeds 10% of free system memory.
2022-02-20 09:51:58.976302: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102252544 exceeds 10% of free system memory.


  1/775 [..............................] - ETA: 28:14 - loss: 0.6999 - accuracy: 0.4375

2022-02-20 09:51:59.206870: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102252544 exceeds 10% of free system memory.


775/775 [==============================] - 600s 772ms/step - loss: 0.4894 - accuracy: 0.7539 - val_loss: 0.1653 - val_accuracy: 0.9375
Epoch 2/20


...

2022-02-20 10:21:46.800179: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


775/775 [==============================] - 615s 794ms/step - loss: 0.2076 - accuracy: 0.9227 - val_loss: 0.6008 - val_accuracy: 0.9375
Epoch 5/20


2022-02-20 10:32:01.838991: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


775/775 [==============================] - 591s 761ms/step - loss: 0.1963 - accuracy: 0.9276 - val_loss: 0.4117 - val_accuracy: 0.9375


2022-02-20 10:41:53.115118: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 158, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 79, 79, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 77, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 36, 32)        9248      
                                                                 
 dropout (Dropout)           (None, 36, 36, 32)        0

Save the model file

In [11]:
model.save('malaria_two.h5')

### Evaluate the model on unseen data with our test set 

In [12]:
model.evaluate(test)

87/87 [==============================] - 15s 164ms/step - loss: 0.2094 - accuracy: 0.9525


[0.2093624770641327, 0.9524673223495483]

Here we see that although our loss is moderately high because of the low number of samples, we got 95.2% accuracy on the test data, which is very good for a CNN trained on a dataset of just around 25k images.